In [0]:
%tensorflow_version 2.x

In [0]:
#IMPORT
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Activation, Flatten, Dropout, MaxPooling2D, Conv2D
from tensorflow.keras.activations import relu, softmax,sigmoid
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.metrics import Accuracy
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from sklearn.metrics import classification_report

In [0]:
def Edadataset(Datasetname,Indexno,FromTrainortest):
  (train_input, train_output),(test_input, test_output) = Datasetname.load_data()
  print(type(train_input), type(train_output), type(test_input), type(test_output))
  print(train_input.shape, train_output.shape,test_input.shape, test_output.shape)
  if str(FromTrainortest.upper()=="TRAIN"):
    plt.imshow(train_input[Indexno],cmap="gray")
  else:
    plt.imshow(test_input[Indexno],cmap="gray")
  switcher = { 
        0: "T-shirt/top", 
        1: "Trouser", 
        2: "Pullover",
        3: "Dress",
        4: "Coat",
        5: "Sandal",
        6: "Shirt",
        7: "Sneaker",
        8: "Bag",
        9: "Ankle boot",
    } 
  if str(FromTrainortest.upper()=="TRAIN"):
    print(switcher.get(train_output[Indexno], "nothing")) 
  else:
    print(switcher.get(test_output[Indexno], "nothing")) 

In [0]:
(train_input, train_output),(test_input, test_output) = fashion_mnist.load_data()
Final_train_input=train_input.reshape((train_input.shape[0], 28, 28, 1))
Final_train_input = Final_train_input / 255
Final_train_output= to_categorical(train_output)
Final_test_input=test_input.reshape((test_input.shape[0], 28, 28, 1))
Final_test_input = Final_test_input / 255
Final_test_output = to_categorical(test_output)
labelNames = ["top", "trouser", "pullover", "dress", "coat","sandal", "shirt", "sneaker", "bag", "ankle boot"]
Num_Epoch = 30
Lr_Rate = 1e-2
Batch_Size = 32

In [0]:
def FashionModel():
 model=Sequential()
 model.add(Conv2D(32, kernel_size=(3,3), input_shape=(28,28,1),padding="same"))
 model.add(Activation("relu"))
 model.add(Conv2D(32, kernel_size=(3,3),padding="same"))
 model.add(Activation("relu"))
 model.add(MaxPooling2D(pool_size=(2,2)))
 model.add(Dropout(0.25))
 model.add(Conv2D(64, kernel_size=(3,3),padding="same"))
 model.add(Activation("relu"))
 model.add(Conv2D(64, kernel_size=(3,3),padding="same"))
 model.add(Activation("relu"))
 model.add(MaxPooling2D(pool_size=(2,2)))
 model.add(Dropout(0.25))
 model.add(Flatten())
 model.add(Dense(512,activation=relu))
 model.add(Dropout(0.5))
 model.add(Dense(10,activation=softmax))
 opt = SGD(lr=Lr_Rate, momentum=0.9, decay=Lr_Rate / Num_Epoch)
 model.compile(optimizer=opt,loss=categorical_crossentropy,metrics=["acc"])
 return model

## TENSORBOARD


In [0]:
%load_ext tensorboard
tbcallbk = tf.keras.callbacks.TensorBoard()
stopMontior = tf.keras.callbacks.EarlyStopping(monitor="val_loss",mode="auto",verbose=1,patience=10)
%tensorboard --logdir logs

In [0]:
def ClassificationModel():
 model=FashionModel()
 model_history=model.fit(Final_train_input,Final_train_output, Batch_Size, Num_Epoch,callbacks=[tbcallbk,stopMontior],validation_data=(Final_test_input,Final_test_output))
 train_loss = model_history.history['loss']
 val_loss   = model_history.history['val_loss']
 acc = model_history.history['acc']
 val_acc = model_history.history['val_acc']
 with open("loss_history.txt", "w") as text_file:
  for i in range(len(train_loss)):
    output = "Loss :"+ str(train_loss[i]) + " Acc :" + str(acc[i]) + " ValLoss :" + str(val_loss[i]) + " ValAcc :" + str(val_acc[i])
    text_file.write("EPOCH :- "+str(int(i+1))+"  " + output + "\n")
 preds = model.predict(Final_test_input)
 print(classification_report(Final_test_output.argmax(axis=1), preds.argmax(axis=1),target_names=labelNames))

##SAVE MODEL

In [0]:
model.save("Fashinmnist_prediction.model")
! mkdir -p save_model
model.save_weights("save_model/FashionMnist_weights") 
with open("save_model/Fashionmnist_model_json.json", "w") as model_json:
  model_json.write(model.to_json())